# Quickstart Guide for Strands Agents

Strands Agents is a powerful framework for building AI agents that can interact with AWS services and perform complex tasks. This quickstart guide will help you get started with creating your first Strands agent.

Lets get started !


## Creating Your First Agent

Lets get an overview of the agentic components needed.

### 1. Create a simple Agent:

This will create an agent with the default model provider, [Amazon Bedrock](https://aws.amazon.com/bedrock/), and the default model, Claude 3.7 Sonnet, in the region of your AWS setup. While the agent runs in the same local environment as it is being invoked, Amazon Bedrock models will run in an AWS account and your agent will invoke the model in the cloud account. The architecture looks as following:
<center>
    <img src="./images/simple_agent.png" width="75%" />
</center>



In [1]:
import warnings
warnings.filterwarnings(action="ignore", message=r"datetime.datetime.utcnow") 

from strands import Agent
# Initialize your agent
agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    system_prompt="You are a helpful assistant that provides concise responses."
)

# Send a message to the agent
response = agent("Hello! Tell me a joke.")

Why did the scarecrow win an award? Because he was outstanding in his field!

### 2. Add Tools to the Agent:

The [strands-agents-tools](https://github.com/strands-agents/tools) repository provides some in-built tools which you can import. You can also create custom tools using the `@tool` decorator. We can create agents with built-in and custom tools. For instance, adding the built-in tool of a calculator and a custom tool for getting the weather you get the following architecture:
<center>
    <img src="./images/agent_with_tools.png" width="75%" />
</center>
Implementing this architecture you have the following:

In [2]:
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather.")

response = agent("What is the weather today?")
print(response)

I can check the current weather for you. Let me do that right away.
Tool #1: weather
Based on the information I received, the weather today is sunny. It's a great day to spend some time outdoors if you have the opportunity!Based on the information I received, the weather today is sunny. It's a great day to spend some time outdoors if you have the opportunity!



### Invoking tool directly

For some applications it is important to directly call the tool. For instance, you might want to debug the tool, pre-populate the agent knowledge with your customer's information or using a tool inside of another tool. In Strands you can do it using the ``tool`` method of your agent followed by the tool name

In [3]:
# Alternatively, you can invoke the tool directly like so:
agent.tool.calculator(expression="sin(x)", mode="derive", wrt="x", order=2)

╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭─────────────────┬───────────────────────────╮                                                                │
│  │ Operation       │ Calculate 2-th Derivative │                                                                │
│  │ Input           │ sin(x)                    │                                                                │
│  │ Result          │ -sin(x)                   │                                                                │
│  │ With respect to │ x                         │                                                                │
│  ╰─────────────────┴───────────────────────────╯                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

{'status': 'success',
 'content': [{'text': 'Result: -sin(x)'}],
 'toolUseId': 'tooluse_calculator_463845579'}


### 4. Model Provider

The default model provider is [Amazon Bedrock](https://aws.amazon.com/bedrock/) and the default model is Claude 3.7 Sonnet in the region of your current AWS environment

You can specify a different model in Amazon Bedrock providing the model ID string directly:

In [7]:
from strands import Agent

agent = Agent(model="anthropic.claude-3-5-haiku-20241022-v1:0")
print(agent.model.config)

{'model_id': 'anthropic.claude-3-5-haiku-20241022-v1:0'}



For more control over the model configuration, you can create a `BedrockModel` provider instance:

In [9]:
import boto3
from strands import Agent
from strands.models import BedrockModel

# Create a BedrockModel
bedrock_model = BedrockModel(
    model_id="anthropic.claude-3-5-haiku-20241022-v1:0",
    region_name='us-west-2',
    temperature=0.3,
)

agent = Agent(model=bedrock_model)
print(agent.model.config)

{'model_id': 'anthropic.claude-3-5-haiku-20241022-v1:0', 'temperature': 0.3}



More details on the available model providers on the [Model Provider Quickstart page](https://strandsagents.com/0.1.x/user-guide/quickstart/#model-providers)


**Congratulations !! Now you have learned how to build a simple agent using Strands!!**

## [Optional] Lets Build a Task-Specific Agent - RecipeBot 🍽️

Lets create a practical example of a task-specific agent. We create a `RecipeBot` that recommends recipes and answers any cooking related questions. Lets dive in !!

Here's what we will create :

<center>
    <img src="./images/interactive_recipe_agent.png" width="75%" />
</center>

In [10]:
from strands import Agent, tool
from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException
import logging

# Configure logging
logging.getLogger("strands").setLevel(logging.INFO)

# Define a websearch tool
@tool
def websearch(keywords: str, region: str = "us-en", max_results: int | None = None) -> str:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DDGSException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"


In [11]:
# Create a recipe assistant agent
recipe_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    system_prompt="""You are RecipeBot, a helpful cooking assistant.
    Help users find recipes based on ingredients and answer cooking questions.
    Use the websearch tool to find recipes when users mention ingredients or to
    look up cooking information.""",
    # Import the websearch tool we created above
    tools=[websearch],
)

In [12]:
response = recipe_agent("Suggest a recipe with chicken and broccoli.")

# Other examples:
# response = recipe_agent("How do I cook quinoa?")
# response = recipe_agent("How can I substitute white wine in shrimp pasta?")
# response = recipe_agent("What are the health benefits of asparagus?")
print(response)

I'd be happy to find some chicken and broccoli recipes for you! Let me search for some options.
Tool #1: websearch
Based on my search, I found a delicious chicken and broccoli stir-fry recipe that's quick and easy to make. Here it is:

# Chicken and Broccoli Stir-Fry

**Ingredients:**
- 1 pound boneless skinless chicken thighs, cut into 3/4-inch pieces
- 3 tablespoons cornstarch (divided: 1 tbsp for coating chicken, 2 tbsp for sauce)
- 2 tablespoons cooking oil
- 3/4 cup thinly sliced onion
- 2 teaspoons minced fresh ginger
- 4 garlic cloves, thinly sliced
- 1 large head of broccoli, cut into florets
- 1/4 cup water
- 1/4 cup soy sauce
- 2 tablespoons oyster sauce
- 1 tablespoon light brown sugar
- 1/4 teaspoon crushed red pepper flakes (adjust to taste)
- Black pepper to taste

**Instructions:**
1. In a medium bowl, toss chicken pieces with 1 tablespoon cornstarch until coated.

2. Heat oil in a large non-stick skillet over medium-high heat until shimmery (about 2 minutes).

3. Add ch

#### And that's it! We now have a running usecase agent with tools in just a few lines of code 🥳.

For more detailed quickstart guide, check out the [Strands documentation](https://strandsagents.com/0.1.x/user-guide/quickstart/).